In [5]:
import requests
import xmltodict
import datetime
from dateutil import tz
import pytz
import xmltodict
from astropy.time import Time
import ephem
import PES_secrets

In [4]:
entry = requests.get("https://targettool.aavso.org/TargetTool/api/v1/telescope",auth=(PES_secrets.apikey,"api_token"))
print(entry.json())


{'latitude': -35.351835, 'longitude': 149.030122, 'targetaltitude': 20.0, 'sunaltitude': -5.0}


In [9]:
# lets now consider the night in question
# Make an observer
wombat = ephem.Observer()

# Set the date and time 
wombat.date = "2024-06-28"

# Location 
wombat.lon = str(148.12899)
wombat.lat = str(-35.282)

# Elevation 
wombat.elev = 500

# To get U.S. Naval Astronomical Almanac values, use these settings
wombat.pressure = 0
wombat.horizon = '-0:34'

# Calculate sunrise, solar noon, and sunset
sunrise = wombat.next_rising(ephem.Sun())
#noon = wombat.next_transit(ephem.Sun(), start=sunrise)
sunset = wombat.next_setting(ephem.Sun())

# Relocate the horizon to get twilight times
wombat.horizon = '-12'  # -6=civil twilight, -12=nautical, -18=astronomical
beg_twilight = wombat.next_rising(ephem.Sun(), use_center=True)
end_twilight = wombat.next_setting(ephem.Sun(), use_center=True)

# Print the results
#print(f"Noon: {noon}")
print(f"Sunset: {sunset}")
print(f"End nautical twilight: {end_twilight}")
print(f"Begin nautical twilight: {beg_twilight}")
print(f"Sunrise: {sunrise}")
print(f"MJD start: {(end_twilight+2415020-2400000.5 ):.6f}")
print(f"MJD end: {(beg_twilight+2415020-2400000.5 ):.6f}")

Sunset: 2024/6/28 07:04:40
End nautical twilight: 2024/6/28 08:04:42
Begin nautical twilight: 2024/6/28 20:17:01
Sunrise: 2024/6/28 21:17:01
MJD start: 60489.336598
MJD end: 60489.845148


In [6]:
obs_sections = ['eb'] #ac,ep,cv,eb,spp,lpv,yso,het,misc,all.

In [8]:
entry = requests.get("https://targettool.aavso.org/TargetTool/api/v1/targets",\
                     auth=(PES_secrets.apikey,"api_token"),params={'observable':['True'],'obs_section':[obs_sections]})
targets = entry.json()

In [ ]:
targets['targets']
for star_info in targets['targets']:
    star_name = star_info.get('star_name')
    if star_name:
        print(f"Star Name: {star_name}")
        # for each star name look up vsx to find the ephemeris
        fmt_star_name = star_name.replace(" ","+")
        vsx_req = requests.get(f"https://www.aavso.org/vsx/index.php?view=query.votable&ident={fmt_star_name}&format=xml")
        json_data = xmltodict.parse(vsx_req.content)
        const = json_data['VOTABLE']['RESOURCE']['TABLE']['DATA']['TABLEDATA']['TR']['TD'][2]
        radec = json_data['VOTABLE']['RESOURCE']['TABLE']['DATA']['TABLEDATA']['TR']['TD'][3]
        vartyp = json_data['VOTABLE']['RESOURCE']['TABLE']['DATA']['TABLEDATA']['TR']['TD'][4]
        vmin = json_data['VOTABLE']['RESOURCE']['TABLE']['DATA']['TABLEDATA']['TR']['TD'][5]
        vmax = json_data['VOTABLE']['RESOURCE']['TABLE']['DATA']['TABLEDATA']['TR']['TD'][7]
        photsys = json_data['VOTABLE']['RESOURCE']['TABLE']['DATA']['TABLEDATA']['TR']['TD'][6]
        epoch = float(json_data['VOTABLE']['RESOURCE']['TABLE']['DATA']['TABLEDATA']['TR']['TD'][9])
        period = float(json_data['VOTABLE']['RESOURCE']['TABLE']['DATA']['TABLEDATA']['TR']['TD'][11])
        #print(f"{const}, {radec}, {vartyp}, {vmin}, {vmax}, {epoch}, {period}")
        # skip those without an ephemeris
        if (epoch==0 or period==0):
            #print(f"{epoch} {period}")
            continue
        # for each star we check the time of minimum at this date
        # to do so we take the epoch zero and add multiples of the period
        # until the date is near the night of observation
        mjd_start = (end_twilight+2415020-2400000.5 )
        mjd_end = (beg_twilight+2415020-2400000.5 )
        ep = epoch
        while (ep<mjd_start):
            ep += period
        ep -= 0.5 # somewhere there is a jd and not a mjd
        #print(f"Minimum at: {ep}")
        if (ep > mjd_start and ep < mjd_end):
            print('well placed for observing tonight')
            jd_value = ep + 2400000.5 ### this is correct

            # Create an astropy Time object from JD
            t = Time(jd_value, format='jd')
            # Convert to datetime (UTC)
            utc_datetime = t.to_datetime(tz.UTC)
            #print(f"UTC Date and Time: {utc_datetime}")
            target_timezone = pytz.timezone('Australia/Canberra')
            local_datetime = utc_datetime.astimezone(target_timezone)
            print(f"Local datetime (Canberra): {local_datetime.isoformat()}")

Star Name: ASAS J081631-3314.7
Star Name: AW Col
Star Name: AZ Cir
Star Name: BB Cha
Star Name: BP Hyi
Star Name: BV Ant
Star Name: CU Mus
Star Name: Dauban V239
Star Name: Dauban V241
well placed for observing tonight
Local datetime (Canberra): 2024-06-29T01:10:39.360161+10:00
Star Name: Dauban V242
Star Name: Dauban V243
Star Name: del Vel
Star Name: DK Cir
Star Name: DK Pyx
Star Name: DN Pyx
Star Name: DV Gru
Star Name: EN Sge
Star Name: EQ Cru
Star Name: EW Cru
well placed for observing tonight
Local datetime (Canberra): 2024-06-28T22:49:33.889281+10:00
Star Name: FP Car
Star Name: GSC 00814-00323
Star Name: HD 224401
Star Name: IQ Pup
Star Name: KP Eri
Star Name: KP Vir
Star Name: LL Aqr
Star Name: NSV 17304
Star Name: NSV 18207
Star Name: NSV 25882
well placed for observing tonight
Local datetime (Canberra): 2024-06-29T02:10:29.280138+10:00
Star Name: OO Lup
Star Name: OT Lup
well placed for observing tonight
Local datetime (Canberra): 2024-06-28T21:04:25.823597+10:00
Star Name: 